# CBMC-Compatible Harness Generation System - Part 5: Graph Assembly and Testing

This part assembles the LangGraph workflow, connects all nodes, and tests the system with example code.

In [1]:
# Build the graph
workflow = StateGraph(HarnessGenerationState)

# Add nodes
workflow.add_node("frontend", frontend_node)
workflow.add_node("code_embedding", code_embedding_node)
workflow.add_node("analyzer", analyzer_node)
workflow.add_node("junction", junction_node)
workflow.add_node("generator", generator_node)
workflow.add_node("cbmc", cbmc_node)
workflow.add_node("output", output_node)
workflow.add_node("tool_execution", execute_tools)

# Connect the nodes
workflow.add_edge(START, "frontend")
workflow.add_edge("frontend", "code_embedding")
workflow.add_edge("code_embedding", "analyzer")

# Add conditional routing for tool execution
workflow.add_conditional_edges(
    "analyzer",
    route_based_on_tool_use,
    {
        "tool_execution": "tool_execution",
        "continue": "junction"
    }
)
workflow.add_edge("tool_execution", "analyzer")

workflow.add_edge("junction", "generator")

# Add conditional routing for generator tool execution
workflow.add_conditional_edges(
    "generator",
    route_based_on_tool_use,
    {
        "tool_execution": "tool_execution",
        "continue": "cbmc"
    }
)
workflow.add_edge("tool_execution", "generator")

workflow.add_edge("cbmc", "output")
workflow.add_edge("output", END)

# Compile the graph
app = workflow.compile()

In [2]:
# Visualize the graph
Image(app.get_graph().draw_mermaid_png())

In [3]:
# Test the system with our example C code
sample_code = """
#include <stdlib.h>
#include <string.h>

void* allocate_buffer(size_t size) {
    void* buffer = malloc(size);
    if (!buffer) {
        return NULL;
    }
    return buffer;
}

char* duplicate_string(const char* str) {
    size_t len = strlen(str);
    char* new_str = (char*)malloc(len + 1);
    if (!new_str) {
        return NULL;
    }
    strcpy(new_str, str);
    return new_str;
}

int process_data(void* data, size_t size) {
    int* processed = (int*)malloc(sizeof(int) * size);
    if (!processed) {
        return -1;
    }
    
    // Process data
    for (size_t i = 0; i < size; i++) {
        processed[i] = ((int*)data)[i] * 2;
    }
    
    // Check condition - potential memory leak
    if (size > 100) {
        return 1; // Memory leak: forgot to free processed
    }
    
    free(processed);
    return 0;
}
"""

In [4]:
# Execute the workflow
initial_message = HumanMessage(content=f"""
I need to analyze the following C code for memory leaks and generate verification harnesses:

```c
{sample_code}
```

Please identify any potential memory leaks and generate CBMC harnesses for verification.
""")

# Run the workflow
result = app.invoke({
    "messages": [initial_message],
    "source_code": sample_code,
    "embeddings": {},
    "vulnerable_functions": [],
    "harnesses": {},
    "cbmc_results": {},
})

# Display the conversation
print("=== Workflow Execution Results ===")
for i, message in enumerate(result["messages"]):
    if isinstance(message, HumanMessage):
        print(f"\n===== Human Message {i+1} =====")
        print(message.content[:200] + "..." if len(message.content) > 200 else message.content)
    elif isinstance(message, AIMessage):
        print(f"\n===== AI Message {i+1} =====")
        print(message.content)
    elif isinstance(message, ToolMessage):
        print(f"\n===== Tool Message {i+1} =====")
        print(message.content[:200] + "..." if len(message.content) > 200 else message.content)

=== Workflow Execution Results ===

===== Human Message 1 =====

I need to analyze the following C code for memory leaks and generate verification harnesses:

```c
#include <stdlib.h>
#include <string.h>

void* allocate_buffer(size_t size) {
    void...

===== AI Message 2 =====
Received source code (682 characters). Proceeding with code embedding and analysis.

===== AI Message 3 =====
Source code embedded successfully. Found 3 functions.

===== AI Message 4 =====
Analysis complete. Identified 3 potentially vulnerable functions: allocate_buffer, duplicate_string, process_data

===== AI Message 5 =====
Preparing to generate harnesses for each vulnerable function.

===== AI Message 6 =====
Generated harnesses for 3 functions.

===== AI Message 7 =====
CBMC Verification Results:

Function: allocate_buffer
Status: WARNING
Message: VERIFICATION WARNING: Allocation function - harness should verify proper allocation and check for NULL.
Suggestions: Add appropriate assertions to verify alloca

In [5]:
# Visualize the architecture with a Mermaid diagram
mermaid_code = """
flowchart TD
    subgraph LangGraph["LangGraph Workflow Orchestrator"]
        SourceCode["Source Code"] -.-> Frontend["Frontend"]
        Frontend -->|Source Code| CodeEmbedding["Code Embedding System"]
        CodeEmbedding -->|MCP| Analyzer["Analyzer (LLM Agent)"]
        Analyzer -->|"dot each"| Junction["Junction"]
        Junction --> G["Generator (LLM Agent)"]
        G -->|Harness| CBMC["CBMC"]
        CBMC -->|Feedback| G
        G -->|Results| Output["Output"]
        
        %% Database connections
        subgraph KnowledgeBase["Knowledge Base"]
            PatternDB[(Pattern Collection<br/>ChromaDB)]
        end
        
        subgraph CodeEmbeddingSystem
            CodeDB[(Source Code Collection<br/>ChromaDB)]
        end
        
        %% Connect to Knowledge Base
        Analyzer <-->|MCP| KnowledgeBase
        G <-->|MCP| KnowledgeBase
        
        %% Connect source code to CBMC
        SourceCode -.-> CBMC
        
        %% Connect Generator to Code Embedding System
        G <-->|MCP| CodeEmbeddingSystem
        Analyzer <-->|MCP| CodeEmbeddingSystem
    end
    
    classDef component fill:#f9f,stroke:#333,stroke-width:2px;
    classDef data fill:#bbf,stroke:#333,stroke-width:1px;
    classDef embedding fill:#bfb,stroke:#333,stroke-width:2px;
    classDef workflow fill:#fff,stroke:#f66,stroke-width:3px;
    classDef kb fill:#fcf,stroke:#333,stroke-width:2px;
    
    class Frontend,Analyzer,Junction,G,CBMC component;
    class SourceCode,Output data;
    class CodeEmbedding,CodeDB embedding;
    class KnowledgeBase kb;
    class PatternDB kb;
    class LangGraph workflow;

    %% MCP connections shown with solid lines
    %% Dotted "dot each" line for functions with memory leak potential
    %% LangGraph surrounds and orchestrates the entire workflow
"""

from IPython.display import display, Markdown
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

```mermaid
flowchart TD
    subgraph LangGraph["LangGraph Workflow Orchestrator"]
        SourceCode["Source Code"] -.-> Frontend["Frontend"]
        Frontend -->|Source Code| CodeEmbedding["Code Embedding System"]
        CodeEmbedding -->|MCP| Analyzer["Analyzer (LLM Agent)"]
        Analyzer -->|"dot each"| Junction["Junction"]
        Junction --> G["Generator (LLM Agent)"]
        G -->|Harness| CBMC["CBMC"]
        CBMC -->|Feedback| G
        G -->|Results| Output["Output"]
        
        %% Database connections
        subgraph KnowledgeBase["Knowledge Base"]
            PatternDB[(Pattern Collection<br/>ChromaDB)]
        end
        
        subgraph CodeEmbeddingSystem
            CodeDB[(Source Code Collection<br/>ChromaDB)]
        end
        
        %% Connect to Knowledge Base
        Analyzer <-->|MCP| KnowledgeBase
        G <-->|MCP| KnowledgeBase
        
        %% Connect source code to CBMC
        SourceCode -.-> CBMC
        
        %% Connect Generator to Code Embedding System
        G <-->|MCP| CodeEmbeddingSystem
        Analyzer <-->|MCP| CodeEmbeddingSystem
    end
    
    classDef component fill:#f9f,stroke:#333,stroke-width:2px;
    classDef data fill:#bbf,stroke:#333,stroke-width:1px;
    classDef embedding fill:#bfb,stroke:#333,stroke-width:2px;
    classDef workflow fill:#fff,stroke:#f66,stroke-width:3px;
    classDef kb fill:#fcf,stroke:#333,stroke-width:2px;
    
    class Frontend,Analyzer,Junction,G,CBMC component;
    class SourceCode,Output data;
    class CodeEmbedding,CodeDB embedding;
    class KnowledgeBase kb;
    class PatternDB kb;
    class LangGraph workflow;

    %% MCP connections shown with solid lines
    %% Dotted "dot each" line for functions with memory leak potential
    %% LangGraph surrounds and orchestrates the entire workflow
```

## CBMC-Compatible Harness Generation System: Summary

This notebook demonstrates a complete implementation of the architecture shown in the diagram, using LangGraph to orchestrate the workflow. The system:

1. **Accepts source code** through the frontend node
2. **Embeds the code** using ChromaDB, extracting functions and their properties
3. **Analyzes potential vulnerabilities** using a specialized LLM agent that identifies memory leak patterns
4. **Queries a pattern database** (also in ChromaDB) to match against known memory leak patterns
5. **Generates CBMC-compatible harnesses** for each vulnerable function
6. **Executes CBMC verification** to identify actual memory leaks
7. **Provides a comprehensive report** with results and suggestions

### Key Components:

- **ChromaDB** for vector storage of both source code and memory leak patterns
- **LangGraph** for orchestrating the complex multi-step workflow
- **Claude LLM** for specialized code analysis and harness generation
- **Message-based communication** between components
- **Memory Leak Pattern Detection** using embedding similarity and metadata

This implementation successfully reproduces the architecture in the provided diagram, with all components working together through the LangGraph orchestration framework.